In [1]:
# !pip install geojson geopandas osmnx spektral matplotlib==3.1.3

# from google.colab import files
# from google.colab import drive

# drive.mount('/content/drive')

## General Import

In [2]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from math import sqrt

import gc
import time
import seaborn as sns; sns.set()


In [3]:
import geojson
import geopandas as gpd
from fiona.crs import from_epsg
import os, json
from shapely.geometry import shape, Point, Polygon, MultiPoint
from geopandas.tools import sjoin
import matplotlib.cm as cm
import matplotlib.pyplot as plt # plotting
import seaborn as sns; sns.set()
from IPython.display import Image

import pickle
import folium


from branca.colormap import  linear
import json
import branca.colormap as cm

from tqdm.notebook import tqdm

import gc


# Visualize Streets Network

In [4]:
path_feat_flow = 'drive/MyDrive/Colab Notebooks/MTL traffic forecasting/bxl graph/FEATURES_FLOW.csv'
path_feat_vel = 'drive/MyDrive/Colab Notebooks/MTL traffic forecasting/bxl graph/FEATURES_VEL.csv'


# flow
features_flow = pd.read_csv(path_feat_flow).iloc[:,1:].values

# vel
features_vel = pd.read_csv(path_feat_vel).iloc[:,1:].values

In [5]:
new_table = features_vel #np.concatenate([features_flow[:,:-1], features_vel], axis=1)

# Prepare Data For Multistep-ahead Forecasting, Split in Training and Testing Sets

In [6]:
def split_sequences_multistep(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)

        del seq_x, seq_y


    del sequences
    gc.collect()

    return np.array(X), np.array(y)

In [7]:
# transform series into train and test sets for supervised learning

def data_mimo_persistent(dataframe, n_train, n_lag, n_output):
    # extract raw values
    raw_values = dataframe#.values
    # transform into supervised learning problem X, y
    X,Y = split_sequences_multistep(raw_values, n_lag, n_output)
    X_tr, Y_tr, X_val, Y_val = X[:-n_train+23], Y[:-n_train+23], X[-n_train+23:],Y[-n_train+23:]

    del X, Y, dataframe
    gc.collect()

    return X_tr, Y_tr, X_val, Y_val

In [8]:
n_LAGS = 12
n_OUTPUT = 12
granularity = 2*2

    
DATAFRAME = new_table


nRow, nCol = DATAFRAME.shape
print(f'Consider {nRow} instances (rows) and {nCol} streets segments (columns)')
print('')

n_TRAIN = 168*2*granularity # 2 weeks

X_tr_pers, Y_tr_pers, X_val_pers, Y_val_pers = data_mimo_persistent(DATAFRAME, n_TRAIN, n_LAGS, n_OUTPUT)

datetime = Y_val_pers[:,:,-1]

x_tr_pers = X_tr_pers[:,:,:-1].astype(np.float32)
y_tr_pers = Y_tr_pers[:,:,:-1].astype(np.float32)
x_val_pers = X_val_pers[:,:,:-1].astype(np.float32)
y_val_pers = Y_val_pers[:,:,:-1].astype(np.float32)

print('X_Train: %s, Y_Train: %s' % (x_tr_pers.shape, y_tr_pers.shape))
print('')
print('X_Valid: %s, Y_Valid: %s' % (x_val_pers.shape, y_val_pers.shape))

Consider 5664 instances (rows) and 4525 streets segments (columns)

X_Train: (4320, 12, 4524), Y_Train: (4320, 12, 4524)

X_Valid: (1321, 12, 4524), Y_Valid: (1321, 12, 4524)


In [9]:
del features_flow, features_vel, DATAFRAME, new_table, y_tr_pers
gc.collect()

52

# Function defining Seasonal Persistence Model

In [10]:
def seasonal_mean(X, seas, n_seq):

    # seas_2 = 168*2*2*2
    seas_3 = 168*2*2

    list_seas_2 = []
    list_seas_3 = []
    
    if seas > X.shape[0]:
        
        for i in reversed(range(1, 24*2 +1)):
            season = X[-i][-1].astype(np.int32)
            list_seas.append(season)
    else:
        
        # for i in reversed(range(seas_3+1,seas_2+1)):
        #     season_2 = X[-i][-1].astype(np.int32)
        #     list_seas_2.append(season_2)
            
        for i in reversed(range(1,seas_3+1)):
            season_3 = X[-i][-1]
            list_seas_3.append(season_3)
            
        list_seas = np.array([
                              # np.vstack(list_seas_2),
                              np.vstack(list_seas_3)]).mean(axis=0)

    del list_seas_3, season_3, X
    gc.collect()
            
    return list_seas[:n_seq]

# Test Model

In [11]:
import math


def evaluate_forecasts(targets, forecasts, n_seq):
    
    list_rmse = []
    list_mae = []
    
    for i in range(n_seq):
        
        true = np.vstack([target[i] for target in targets])
        predicted = np.vstack([forecast[i] for forecast in forecasts])
        
        rmse = np.sqrt((np.square(true - predicted)).mean(axis=0))
        mae = np.absolute(true - predicted).mean(axis=0)
        
        list_rmse.append(rmse)
        list_mae.append(mae)
        
    list_rmse = np.vstack(list_rmse)
    list_mae = np.vstack(list_mae)
    
    return list_rmse, list_mae

In [12]:
seas = 168*2*2*2 # weekly season - past 4 weeks 

forecasts = []
targets = []

rmse_list = []
mae_list = []

img_list = []


for i in tqdm(range(len(y_val_pers))):

    
    x_tr_pers = np.insert(x_tr_pers, x_tr_pers.shape[0], x_val_pers[i], axis=0)
    Y = y_val_pers[i]
    
    # make forecast
    forecast = seasonal_mean(x_tr_pers, seas, n_OUTPUT).astype(np.int32)
    
    # retrieve true value
    true_value = Y.astype(np.int32)

    # print('step '+str(i))
    # print(datetime[i][0])
    # print('* Prediction Accuracy (MAE) '+ str(np.absolute(true_value - forecast).mean()))
    # print('----')

    forecasts.append(forecast)
    targets.append(true_value)
    
    # evaluate model performance each period
    rmse, mae = evaluate_forecasts(targets, forecasts, 12)
           
    rmse_list.append(rmse)
    mae_list.append(mae)
    
#     # Forecast Horizons - H = {1, 2, ..., 12}
#     x = ['t+1','t+2','t+3','t+4','t+5','t+6','t+7','t+8','t+9','t+10','t+11','t+12']

#     fig = plt.figure(figsize=(15,7))
#     plt.subplot(311)
#     plt.title('Mean of speed predictions for all highways in Belgium')
#     plt.plot(np.mean(forecast[:, 5975:], axis=1), label='Baseline Prediction') 
#     plt.plot(np.mean(true_value[:, 5975:], axis=1), label='Truth')
#     plt.ylabel('Trucks Flow')
#     plt.ylim(-1, 350000)
# #     plt.xticks(rotation=45)
#     plt.legend()
    
#     plt.subplot(312)
#     plt.plot(np.sum(forecast, axis=1), label='Baseline Prediction') 
#     plt.plot(np.sum(true_value, axis=1), label='Truth')
#     plt.ylabel('Zoom ')
# #     plt.ylim(-1, 150)
# #     plt.xticks(rotation=45)
#     plt.legend()
    
#     plt.subplot(313)
#     plt.errorbar(x, np.absolute(true_value - forecast).mean(axis=1), mae_list[-1].std(axis=1),
#                  fmt='o', color='grey',
#                  ecolor='lightblue', elinewidth=3, capsize=0)
    
#     plt.ylabel('error MAE +- std')
#     plt.xticks(rotation=45)
#     plt.xlabel('Time (t): '+str(datetime[i][-1]))

#     # plt.show()
#     fig.savefig(str(i)+'_plot.png')
# #     img_list.append(fig)
#     fig.clear()
#     plt.close(fig)


#     print('Prediction Accuracy (MAE) '+str(np.absolute(true_value - forecast).mean()))

    
    del forecast, true_value, rmse, mae
    gc.collect()


# # Saving the objects:
# with open('list_plots.pickle', 'wb') as f:
#     pickle.dump([img_list], f)

In [13]:
np.mean(rmse_list)

11.65820625509963

In [14]:
np.mean(mae_list)

5.57054593069183

In [15]:
(18.38+5.87)/2
# (10.88+ 3.68)/2

12.125

In [16]:
RMSE_MEAN = np.mean(rmse_list,axis=0).mean(axis=1)
RMSE_STD =  np.std(rmse_list,axis=0).std(axis=1)

for i in range(len(RMSE_MEAN)):
    print('t+'+str(i+1)+' RMSE MEAN ' +str(np.round(RMSE_MEAN[i],3))+' +- '+str(np.round(RMSE_STD[i],3)))
    print('')
    

t+1 RMSE MEAN 11.67 +- 0.701

t+2 RMSE MEAN 11.679 +- 0.714

t+3 RMSE MEAN 11.679 +- 0.721

t+4 RMSE MEAN 11.68 +- 0.734

t+5 RMSE MEAN 11.678 +- 0.745

t+6 RMSE MEAN 11.672 +- 0.747

t+7 RMSE MEAN 11.667 +- 0.759

t+8 RMSE MEAN 11.661 +- 0.764

t+9 RMSE MEAN 11.65 +- 0.769

t+10 RMSE MEAN 11.637 +- 0.77

t+11 RMSE MEAN 11.621 +- 0.761

t+12 RMSE MEAN 11.605 +- 0.758



In [17]:
MAE_MEAN = np.mean(mae_list,axis=0).mean(axis=1)
MAE_STD =  np.std(mae_list,axis=0).std(axis=1)

for i in range(len(MAE_MEAN)):
    print('t+'+str(i+1)+' MAE MEAN ' +str(np.round(MAE_MEAN[i],3))+' +- '+str(np.round(MAE_STD[i],3)))
    print('')
    

t+1 MAE MEAN 5.566 +- 0.676

t+2 MAE MEAN 5.576 +- 0.696

t+3 MAE MEAN 5.58 +- 0.701

t+4 MAE MEAN 5.583 +- 0.719

t+5 MAE MEAN 5.586 +- 0.734

t+6 MAE MEAN 5.584 +- 0.732

t+7 MAE MEAN 5.58 +- 0.742

t+8 MAE MEAN 5.577 +- 0.754

t+9 MAE MEAN 5.57 +- 0.757

t+10 MAE MEAN 5.561 +- 0.763

t+11 MAE MEAN 5.548 +- 0.748

t+12 MAE MEAN 5.535 +- 0.742



In [18]:
with open('vel_HA_RMSE.pkl', 'wb') as f:  
    pickle.dump(rmse_list, f)

!cp vel_HA_RMSE.pkl "drive/MyDrive/Colab Notebooks/MTL traffic forecasting/Results/"
print('RMSE')


with open('vel_HA_MAE.pkl', 'wb') as f:  
    pickle.dump(mae_list, f)

!cp vel_HA_MAE.pkl "drive/MyDrive/Colab Notebooks/MTL traffic forecasting/Results/"
print('MAE')

RMSE
MAE


In [19]:
while True:pass

KeyboardInterrupt: ignored